In [ ]:
# pip install faker mysql-connector-python

Đã tạo file 'dulieu_theo_trinhtu.sql' thành công!
Kịch bản đã được cấu trúc lại để tuân thủ quy trình và định dạng dữ liệu bạn mong muốn.


In [25]:
import random
import datetime
from faker import Faker

# --- CÁC THAM SỐ ---
NUM_HOCSINH = 100
NUM_GIAOVIEN = 24
NUM_GIAOVU = 3
CURRENT_NAMHOC_ID = 'NH2025'
CURRENT_NIENKHOA = 2025
# --------------------

fake = Faker('vi_VN')

# --- DỮ LIỆU CỐ ĐỊNH THEO ĐÚNG MẪU BẠN GỬI ---
VAITRO_DATA = [('VT01', 'Học sinh'), ('VT02', 'Giáo viên'), ('VT03', 'Giáo vụ')]
MONHOC_DATA = [
    ('MH01', 'Toán học'), ('MH02', 'Vật lý'), ('MH03', 'Hóa học'),
    ('MH04', 'Sinh học'), ('MH05', 'Ngữ văn'), ('MH06', 'Lịch sử'),
    ('MH07', 'Địa lý'), ('MH08', 'Tiếng Anh')
]
LOAIDIEM_DATA = [
    ('LD01', 'Miệng', 'Điểm kiểm tra miệng', 1),
    ('LD02', '15 phút', 'Điểm kiểm tra 15 phút', 1),
    ('LD03', '1 tiết', 'Điểm kiểm tra 1 tiết', 2),
    ('LD04', 'Học kỳ', 'Điểm kiểm tra học kỳ', 3)
]
CHUCVU_DATA = [
    ('CV01', 'Lớp trưởng', 'Quản lý lớp học', 'VT01'),
    ('CV02', 'Tổ trưởng', 'Quản lý tổ học tập', 'VT01'),
    ('CV03', 'Giáo viên chủ nhiệm', 'Quản lý và chủ nhiệm lớp', 'VT02'),
    ('CV04', 'Giáo viên bộ môn', 'Giảng dạy các bộ môn', 'VT02'),
    ('CV05', 'Thành viên lớp', 'Học sinh trong lớp', 'VT01'),
    ('CV06', 'Giáo vụ', 'Nhân sư phòng giáo vụ', 'VT03')
]
CV_LOPTRUONG, CV_TOTRUONG, CV_GVCN, CV_GVBM, CV_HOCSINH, CV_GIAOVU = [cv[0] for cv in CHUCVU_DATA]

#--- BƯỚC 1: TẠO DỮ LIỆU TRONG BỘ NHỚ ---

# Lưu trữ các đối tượng đã tạo để tham chiếu chéo
generated_hocsinh = []
generated_giaovien = []
generated_giaovu = []
generated_lop = []

# Lưu trữ các câu lệnh SQL để ghi ra file theo từng bảng
sql = { table: [] for table in [
    "VAITRO", "USERS", "GIAOVIEN", "GIAOVU", "HOCSINH", "MONHOC", "NAMHOC", "LOP", 
    "DIEM", "LOAIDIEM", "CHITIETDIEM", "CHUCVU", "HOSO", "HOSOGIAOVIEN", "HOSOHOCSINH", "HOSOGIAOVU"
]}

# 1.1. Tạo dữ liệu tĩnh
for vtid, tenvt in VAITRO_DATA:
    sql["VAITRO"].append(f"INSERT INTO VAITRO (VaiTroID, TenVaiTro) VALUES ('{vtid}', '{tenvt}');")
for mhid, tenmh in MONHOC_DATA:
    sql["MONHOC"].append(f"INSERT INTO MONHOC (MonHocID, TenMonHoc) VALUES ('{mhid}', '{tenmh}');")
for ldid, tenld, mota, heso in LOAIDIEM_DATA:
    sql["LOAIDIEM"].append(f"INSERT INTO LOAIDIEM (LoaiDiemID, TenLoaiDiem, MoTa, HeSo) VALUES ('{ldid}', '{tenld}', '{mota}', {heso});")
for cvid, tencv, mota, vtid in CHUCVU_DATA:
    sql["CHUCVU"].append(f"INSERT INTO CHUCVU (ChucVuID, TenChucVu, MoTa, VaiTroID) VALUES ('{cvid}', '{tencv}', '{mota}', '{vtid}');")

# 1.2. Tuân thủ trình tự: User -> HoSo -> GiaoVien/HocSinh
user_counter = 1
hoso_counter = 1

# === TẠO HỌC SINH ===
for i in range(1, NUM_HOCSINH + 1):
    hs_id = f"HS{i:06d}"
    user_id = f"U{user_counter:07d}"
    hoso_id = f"HSO{hoso_counter:05d}" # HoSoID riêng biệt
    
    # Lưu đối tượng để dùng sau
    student_info = {'hs_id': hs_id, 'user_id': user_id, 'hoso_id': hoso_id}
    generated_hocsinh.append(student_info)
    
    if i % 40 == 1: chucvu_id = CV_LOPTRUONG
    elif i % 10 == 1: chucvu_id = CV_TOTRUONG
    else: chucvu_id = CV_HOCSINH
    
    gioitinh = random.choice(["Nam", "Nữ"])
    if gioitinh == "Nam":
        hoten = fake.name_male()
    else:
        hoten = fake.name_female()
    ngaysinh = fake.date_of_birth(minimum_age=15, maximum_age=18).strftime('%Y-%m-%d')
    email = f"{hs_id.lower()}@student.com"
    diachi = fake.address().replace('\n', ', ')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    sql["USERS"].append(f"INSERT INTO USERS (UserID, TenDangNhap, MatKhau, VaiTroID) VALUES ('{user_id}', 'hs{i:03d}', '123456', 'VT01');")
    sql["HOSO"].append(f"INSERT INTO HOSO (HoSoID, HoTen, GioiTinh, NgaySinh, Email, DiaChi, ChucVuID, TrangThaiHoSo, NgayTao, NgayCapNhatGanNhat) VALUES ('{hoso_id}', '{hoten}', '{gioitinh}', '{ngaysinh}', '{email}', '{diachi}', '{chucvu_id}', 'Đang hoạt động', '{now}', '{now}');")
    sql["HOCSINH"].append(f"INSERT INTO HOCSINH (HocSinhID, UserID) VALUES ('{hs_id}', '{user_id}');")
    
    user_counter += 1
    hoso_counter += 1

# === TẠO GIÁO VIÊN ===
for i in range(1, NUM_GIAOVIEN + 1):
    gv_id = f"GV{i:06d}"
    user_id = f"U{user_counter:07d}"
    hoso_id = f"HSO{hoso_counter:05d}"
    
    teacher_info = {'gv_id': gv_id, 'user_id': user_id, 'hoso_id': hoso_id}
    generated_giaovien.append(teacher_info)
    
    chucvu_id = CV_GVCN if i <= 12 else CV_GVBM
    gioitinh = random.choice(["Nam", "Nữ"])
    if gioitinh == "Nam":
        hoten = fake.name_male()
    else:
        hoten = fake.name_female()
    ngaysinh = fake.date_of_birth(minimum_age=25, maximum_age=55).strftime('%Y-%m-%d')
    email = f"{gv_id.lower()}@teacher.com"
    diachi = fake.address().replace('\n', ', ')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    sql["USERS"].append(f"INSERT INTO USERS (UserID, TenDangNhap, MatKhau, VaiTroID) VALUES ('{user_id}', 'gv{i:03d}', '123456', 'VT02');")
    sql["HOSO"].append(f"INSERT INTO HOSO (HoSoID, HoTen, GioiTinh, NgaySinh, Email, DiaChi, ChucVuID, TrangThaiHoSo, NgayTao, NgayCapNhatGanNhat) VALUES ('{hoso_id}', '{hoten}', '{gioitinh}', '{ngaysinh}', '{email}', '{diachi}', '{chucvu_id}', 'Đang hoạt động', '{now}', '{now}');")
    sql["GIAOVIEN"].append(f"INSERT INTO GIAOVIEN (GiaoVienID, UserID) VALUES ('{gv_id}', '{user_id}');")
    
    user_counter += 1
    hoso_counter += 1

# === TẠO GIÁO VỤ ===
for i in range(1, NUM_GIAOVU + 1):
    gvu_id = f"AD{i:06d}"
    user_id = f"U{user_counter:07d}"
    hoso_id = f"HSO{hoso_counter:05d}"

    admin_info = {'gvu_id': gvu_id, 'user_id': user_id, 'hoso_id': hoso_id}
    generated_giaovu.append(admin_info)
    
    hoten = fake.name()
    gioitinh = random.choice(["Nam", "Nữ"])
    if gioitinh == "Nam":
        hoten = fake.name_male()
    else:
        hoten = fake.name_female()
    ngaysinh = fake.date_of_birth(minimum_age=30, maximum_age=58).strftime('%Y-%m-%d')
    email = f"{gvu_id.lower()}@edu.com"
    diachi = fake.address().replace('\n', ', ')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    sql["USERS"].append(f"INSERT INTO USERS (UserID, TenDangNhap, MatKhau, VaiTroID) VALUES ('{user_id}', 'gvu{i:03d}', '123456', 'VT03');")
    sql["HOSO"].append(f"INSERT INTO HOSO (HoSoID, HoTen, GioiTinh, NgaySinh, Email, DiaChi, ChucVuID, TrangThaiHoSo, NgayTao, NgayCapNhatGanNhat) VALUES ('{hoso_id}', '{hoten}', 'Nam', '{ngaysinh}', '{email}', '{diachi}', '{CV_GIAOVU}', 'Đang hoạt động', '{now}', '{now}');")
    sql["GIAOVU"].append(f"INSERT INTO GIAOVU (GiaoVuID, UserID) VALUES ('{gvu_id}', '{user_id}');")

    user_counter += 1
    hoso_counter += 1

# 1.3. Tuân thủ trình tự: Lop, Nam Hoc
sql["NAMHOC"].append(f"INSERT INTO NAMHOC (NamHocID, MoTa, BatDau, KetThuc) VALUES ('{CURRENT_NAMHOC_ID}', 'Năm học {CURRENT_NIENKHOA}-{CURRENT_NIENKHOA+1}', '2025-09-05', '2026-05-31');")
lop_ids_map = {f'10A{i}': f'10A{i}' for i in range(1, 3)}
lop_ids_map.update({f'10B{i}': f'10B{i}' for i in range(1, 2)})
lop_ids_map.update({f'10C{i}': f'10C{i}' for i in range(1, 2)})
lop_ids_map.update({f'11A{i}': f'11A{i}' for i in range(1, 3)})
lop_ids_map.update({f'11B{i}': f'11B{i}' for i in range(1, 2)})
lop_ids_map.update({f'11C{i}': f'11C{i}' for i in range(1, 2)})
lop_ids_map.update({f'12A{i}': f'12A{i}' for i in range(1, 3)})
lop_ids_map.update({f'12B{i}': f'12B{i}' for i in range(1, 2)})
lop_ids_map.update({f'12C{i}': f'12C{i}' for i in range(1, 2)})

gvcn_iterator = 0
for ten_lop, lop_id in lop_ids_map.items():
    gvcn_id = generated_giaovien[gvcn_iterator]['gv_id']
    siso = NUM_HOCSINH // len(lop_ids_map)
    sql["LOP"].append(f"INSERT INTO LOP (LopID, TenLop, SiSo, GVCNID) VALUES ('{lop_id}', '{ten_lop}', {siso}, '{gvcn_id}');")
    generated_lop.append({'lop_id': lop_id, 'gvcn_id': gvcn_id})
    gvcn_iterator += 1

# 1.4. Tuân thủ trình tự: Phân bổ và tạo HoSoHocSinh/HoSoGiaoVien/HoSoGiaoVu
hhs_counter = 1
hgv_counter = 1
hgvu_counter = 1
lop_id_list = [lop['lop_id'] for lop in generated_lop]

for i, student in enumerate(generated_hocsinh):
    lop_id_assigned = lop_id_list[i % len(lop_id_list)]
    hhs_id = f"HHS{hhs_counter:03d}"
    sql["HOSOHOCSINH"].append(f"INSERT INTO HOSOHOCSINH (HoSoHocSinhID, HocSinhID, HoSoID, LopHocID, NienKhoa) VALUES ('{hhs_id}', '{student['hs_id']}', '{student['hoso_id']}', '{lop_id_assigned}', {CURRENT_NIENKHOA});")
    hhs_counter += 1

for i, teacher in enumerate(generated_giaovien):
    lop_id_assigned = generated_lop[i%len(generated_lop)]['lop_id']
    hgv_id = f"HGV{hgv_counter:03d}"
    now_str = datetime.datetime.now().strftime('%Y-%m-%d')
    sql["HOSOGIAOVIEN"].append(f"INSERT INTO HOSOGIAOVIEN (HoSoGiaoVienID, GiaoVienID, HoSoID, LopDayID, NgayBatDauLamViec) VALUES ('{hgv_id}', '{teacher['gv_id']}', '{teacher['hoso_id']}', '{lop_id_assigned}', '{now_str}');")
    hgv_counter += 1

for i, admin in enumerate(generated_giaovu):
    hgvu_id =  f"HAD{hgvu_counter:03d}"
    sql["HOSOGIAOVU"].append(f"INSERT INTO HOSOGIAOVU (HoSoGiaoVuID, GiaoVuID, HoSoID) VALUES ('{hgvu_id}', '{admin['gvu_id']}', '{admin['hoso_id']}');")
    hgvu_counter += 1
# 1.5. Tạo dữ liệu điểm
diem_counter = 1
ctd_counter = 1
for student in generated_hocsinh:
    for monhoc in MONHOC_DATA:
        for hocky in [1, 2]:
            diem_id = f"D{diem_counter:04d}"
            scores = [
                {'loai': 'LD01', 'diem': round(random.uniform(4, 10), 1)},
                {'loai': 'LD02', 'diem': round(random.uniform(4, 10), 1)},
                {'loai': 'LD03', 'diem': round(random.uniform(3, 10), 1)},
                {'loai': 'LD04', 'diem': round(random.uniform(3, 10), 1)}
            ]
            
            tong_diem, tong_heso = 0, 0
            heso_map = {ld[0]: ld[3] for ld in LOAIDIEM_DATA}
            for score in scores:
                tong_diem += score['diem'] * heso_map[score['loai']]
                tong_heso += heso_map[score['loai']]
            
            diem_tb = round(tong_diem / tong_heso, 2)
            if diem_tb >= 8.5: xep_loai = 'Giỏi'
            elif diem_tb >= 6.5: xep_loai = 'Khá'
            elif diem_tb >= 5.0: xep_loai = 'Trung bình'
            else: xep_loai = 'Yếu'
            
            sql["DIEM"].append(f"INSERT INTO DIEM (DiemID, HocSinhID, MonHocID, NamHocID, HocKy, DiemTrungBinh, XepLoai) VALUES ('{diem_id}', '{student['hs_id']}', '{monhoc[0]}', '{CURRENT_NAMHOC_ID}', {hocky}, {diem_tb}, '{xep_loai}');")
            
            for score in scores:
                ctd_id = f"CTD{ctd_counter:05d}"
                sql["CHITIETDIEM"].append(f"INSERT INTO CHITIETDIEM (ChiTietDiemID, DiemID, LoaiDiemID, GiaTri) VALUES ('{ctd_id}', '{diem_id}', '{score['loai']}', {score['diem']});")
                ctd_counter += 1
            
            diem_counter += 1

# --- BƯỚC 2: GHI RA FILE THEO ĐÚNG THỨ TỰ MẪU ---

with open('dulieu_theo_trinhtu.sql', 'w', encoding='utf-8') as f:
    f.write("-- TỆP DỮ LIỆU ĐƯỢC TẠO THEO ĐÚNG TRÌNH TỰ VÀ MẪU --\n")
    
    table_order = [
        "VAITRO", "CHUCVU", "LOAIDIEM", "MONHOC", "NAMHOC", "USERS", "HOSO", "GIAOVIEN", "HOCSINH", "GIAOVU", "LOP", "HOSOHOCSINH", "HOSOGIAOVIEN", "HOSOGIAOVU", "DIEM", "CHITIETDIEM"
    ]
    
    for table_name in table_order:
        if sql[table_name]:
            f.write(f"\n-- Dữ liệu cho bảng {table_name}\n")
            f.write("\n".join(sql[table_name]))
            f.write("\n")

    f.write("\n-- HOAN TAT TAO DU LIEU --\n")

print("Đã tạo file 'dulieu_theo_trinhtu.sql' thành công!")
print("Kịch bản đã được cấu trúc lại để tuân thủ quy trình và định dạng dữ liệu bạn mong muốn.")

Đã tạo file 'dulieu_theo_trinhtu.sql' thành công!
Kịch bản đã được cấu trúc lại để tuân thủ quy trình và định dạng dữ liệu bạn mong muốn.


In [5]:
# ngaysinh = fake.date_of_birth(minimum_age=15, maximum_age=18).strftime('%Y-%m-%d')
ngaysinh = "Không"
print(type(ngaysinh))

<class 'str'>
